In [1]:
import DSGRN
from pychomp import *
import datetime
import operator
import functools
from itertools import product
from math import log2

class BlowupGraph2:

    def complex(self):
        return self.dc
        
    def diagram(self):
        return self.digraph.adjacencies

    def closedrightfringe(self,i):
        return any(self.cc.rightfringe(k) for k in self.cc.star({i}))

    def blowupfringe(self,i):
        """
        determine if cell i is in the fringe
        """
        return all(self.closedrightfringe(k) for k in self.oc.simplex(self.dc.dual(i)))

    def NormalVariables(self,s):  # i.e. inessential directions
        """
        return the set of normal variables for a cubical cell s
        """
        shape = self.cc.cell_shape(s)
        return [ d for d in range(0,self.D) if shape & (1 << d) == 0]

    def TangentVariables(self,s):  # i.e. essential directions
        """
        return the set of normal variables for a cubical cell s
        """
        shape = self.cc.cell_shape(s)
        return [ d for d in range(0,self.D) if shape & (1 << d) != 0]

    def RelativePositionVector(self,s,t):
        """
        Return the relative position vector of two cubical cells;
        this is equal to the difference of combinatorial position of two cells,
        regarding vertices to be at (even,even,...) integer coordinates and
        higher dimensional cells to have odd coordinates for their dimensions with
        extent
        """
        return [ y-x for (x,y) in zip(self.cc.barycenter(s),self.cc.barycenter(t))]

    def absorbing(self, cctopcell, d, direction):
        """
        Return True if the codim 1 face of cctopcell 
          collapsed in dimension d in direction "direction"
          is absorbing
        """
        # The cc topcell indexing is not compatible with labelling indexing
        # due to the extra wrapped layer, so we compute the index idx
        coords = self.cc.coordinates(cctopcell)
        # "Wrap layer" boxes have all walls absorbing.
        if any(coords[i] == self.limits[i] for i in range(0,self.D)):
            return True
        idx = sum (  c * self.pv[i] for (i,c) in enumerate(coords) )
        labelbit = 1 << (d + (self.D if direction == -1 else 0))
        return self.labelling[idx] & labelbit != 0

    def flowdirection(self, cctopcell, s, t):
        """
        Given a cubical domain, and a dual-order wall (s,t) (i.e. s is a face of t in cc) 
        return 1 if a proof of transversality from s->t is known valid in dom
        return -1 if a proof of transversality from t->s is known valid in dom
        return 0 otherwise
        """
        #print("flowdirection" + str((cctopcell, s, t))) #debug
        sshape = self.cc.cell_shape(s)
        scoords = self.cc.coordinates(s)
        tshape = self.cc.cell_shape(t)
        tcoords = self.cc.coordinates(t)
        xorshape = sshape ^ tshape # 1's in positions where s is 0 and t is 1.
        absorbing = False
        repelling = False
        #print("Line 40" + str((sshape, scoords, tshape, tcoords, xorshape))) #debug
        for d in range(0,self.D):
            if xorshape & (1 << d):
                #print("  collapsed dimension " + str(d)) #debug
                direction = -1 if scoords[d] > tcoords[d] else 1  # -1 left, 1 right
                #print("  direction " + str(direction)) #debug
                #print("  absorbing -> " + str(self.absorbing(cctopcell, d, direction))) #debug
                # mostra print aborbing when is true (not part of the real code) 
                if self.absorbing(cctopcell, d, direction):
                    absorbing = True
                    #print("  \033[0;34;48m absorbing \033[0;30;48m -> True" ) #debug
                else:
                    repelling = True
                    #print("  \033[0;34;48m repelling \033[0;30;48m -> True" ) #debug
        if absorbing and not repelling:
            return -1
        elif repelling and not absorbing:
            return 1
        else:
            return 0
        

    def RookField(self,t):
        """
        Given a cubical top-cell t, return the rook field vector
        """
        M = { (True,True): 0, (True,False): -1, (False,True): 1, (False,False) : 0 }
        return [ M[self.absorbing(t, d, -1),self.absorbing(t, d, 1)] for d in range(0,self.D)]
    
    def CorrectRookField(self,t):
        ###
        #Given a cubical top-cell t, return the correct rook field vector
        ###
        self.a=std.RookField(t)
        return [-1*i for i in self.a]
    
    ########## funtions below for define rule3         

    def Feedback(self,cycle):
        """
        return {  1   if cycle has an even number of repressors
               { -1   otherwise
        """
        edges = list(zip(cycle[:-1],cycle[1:]))+[(cycle[-1],cycle[0])]
        return functools.reduce(operator.mul, [ 1 if self.network.interaction(i,j) else -1 for (i,j) in edges])

    def CrossingNumber(self, q, cycle):
        edges = list(zip(cycle[:-1],cycle[1:]))+[(cycle[-1],cycle[0])]
        return functools.reduce(operator.add, [ 1 if ((1 if self.network.interaction(i,j) else -1) != q[i]*q[j]) else 0 for (i,j) in edges])

    def RegulationMap(self,s):
        """
        Return a dictionary with key-value pairs
        { (k,v) : s has normal variable k which regulates variable v}
        """
        coords = self.cc.coordinates(s)
        #print(self.cc.boxes())
        #print("RegulationMap")
        #print(s)
        #print(self.closedrightfringe(s))
        #print(coords)
        #print(self.NormalVariables(s))
        #for k in self.NormalVariables(s):
         #   print("normal variable")
          #  print(k)
           # print(self.network.outputs(k))
            #print(coords[k])
            #print(self.network.outputs(k)[coords[k]-1])

        return { k : self.network.outputs(k)[coords[k]-1] for k in self.NormalVariables(s)}

    #def isOpaque(self,s): ### has bug
     #   """
      #  Return True if s is an opaque cubical cell,
       #   i.e. if RegulationMap(s) is a permutation
       # """
       # rmap = self.RegulationMap(s)
       # return set(rmap.keys())==set(rmap.values())

    def doubleEdge(self,S,T): #given two vertices S and T in self.digraph return true if has double edge

        if (S,T) in self.digraph.edges():
            if (T,S) in self.digraph.edges():
                return True
        if (T,S) in self.digraph.edges():
            if (S,T) in self.digraph.edges():
                return True   
        else: return False
    
    def NormalOpaqueness(self,s,i): # return true if in the i-th normal direction we have +1 and -1 
        S=self.dc.dual(s)
        dim1high=[a for a in self.cc.star({s}) if self.cc.cell_dim(a)==1+self.cc.cell_dim(s)]
        leftd=0
        rightd=0
        for t in dim1high:
            T=self.dc.dual(t)
            if self.cc.barycenter(t)[i]==self.cc.barycenter(s)[i]-1:
                if self.doubleEdge(S,T): return True
                if (S,T) in self.digraph.edges(): leftd=-1
                if (T,S) in self.digraph.edges(): leftd=1                
            if self.cc.barycenter(t)[i]==self.cc.barycenter(s)[i]+1:
                if self.doubleEdge(S,T): return True
                if (S,T) in self.digraph.edges(): rightd=+1
                if (T,S) in self.digraph.edges(): rightd=-1
        if leftd*rightd==-1: return True
        else: return False

    def isOpaque(self,s): # return true if s is opaque
        N=self.NormalVariables(s)
        S=self.dc.dual(s)

        return all([self.NormalOpaqueness(s,i) for i in N]) # checking with all normal direction we obtain +1 and -1
        

    def isEquilibriumCell(self,s):
        """
        Return True if s is an equilibrium cell
        """
        if not self.isOpaque(s):
            return False
        for t in self.cc.topstar(s):
            rf = self.RookField(t)
            if any(rf[d] != 0 for d in self.TangentVariables(s)):
                return False
        return True

    def EquilibriumCells(self):
        """
        Returning list of equilibrium cells
        """
        return [ s for s in self.cc if (not self.closedrightfringe(s)) and self.isEquilibriumCell(s) ]

    def CycleDecomposition(self,perm):
        """
        Given a key-value map "perm" defining a permutation,
        return a list of cycles, where each cycle is represented 
        as a list of elements
        e.g.  CycleDecomposition({ 1:1, 3:4, 4:3 }) may return
              [[1],[3,4]]  (or any equivalent answer)
        """
        rmap = dict(perm)
        def ExtractCycle(G):
            cycle = list(G.popitem())
            while cycle[-1] in G: 
                cycle.append(G.pop(cycle[-1]))
            return cycle[:-1]
        cycles = []
        while rmap:
            cycles.append(ExtractCycle(rmap))
        return cycles

    def OpacityCycles(self,s):
        """
        Return list of regulatory cycles for an opaque cubical cell s
        """
        return self.CycleDecomposition(self.RegulationMap(s))

    def Rule0(self):
        """
        Produce all edges, except for edges from real cells into wrap cells
        """
        for edge in self.oc(1):
            (s,t) = self.oc.simplex(edge)
            S = self.dc.dual(s)
            T = self.dc.dual(t)
            if self.blowupfringe(S) or not self.blowupfringe(T):
                self.digraph.add_edge(S,T)
            if self.blowupfringe(T) or not self.blowupfringe(S):
                self.digraph.add_edge(T,S) 
            if not self.blowupfringe(S): #### adding self edge
                self.digraph.add_edge(S,S)
            if not self.blowupfringe(T): #### adding self edge
                self.digraph.add_edge(T,T)

    def Rule1(self):
        """
        Remove edges for which there are transversality theorems from Rule 1
        """
        for edge in self.oc(1):
            #print("Line 87. \033[0;31;48m edge \033[0;30;48m  = " + str(edge)) #debug
            (s,t) = self.oc.simplex(edge)
            S = self.dc.dual(s)
            T = self.dc.dual(t)
            if self.blowupfringe(S) or self.blowupfringe(T):
                continue
            #print("Line 91. (s,t,S,T) = " + str((s,t,S,T))) #debug
            # s is the index of the lower dimensional cell in the cubical complex self.cc
            # t is the index of the higher dimensional cell in the cubical complex self.cc
            # (s,t) corresponds to an edge in the order complex, which is a wall in the dual-order complex.
            #print("\033[0;31;48m edge \033[0;30;48m in order complex = " + str(edge)) #debug
            #print("topstar = " + str(self.cc.topstar(t))) #debug
            flowdata = [self.flowdirection(cctopcell,s,t) for cctopcell in self.cc.topstar(t) ]
            #print("Line 96. flowdata = " + str(flowdata)) #debug
            if all ( k == 1 for k in flowdata ):
                #print("\033[0;34;48m removing edge \033[0;30;48m " + str((T,S)) + " correspond to \033[0;34;48m" + str((t,s))+"\033[0;30;48m") #debug
                self.digraph.remove_edge(T, S)
            #else:
            #    print((t,s))
            #    print("      Edge(" + str(self.cc.barycenter(t)) + ", " + str(self.cc.barycenter(s)) + ")")

            if all ( k == -1 for k in flowdata ):
                #print("\033[0;34;48m removing edge \033[0;30;48m " + str((S,T)) + " correspond to \033[0;34;48m" +  str((s,t))+"\033[0;30;48m") #debug
                self.digraph.remove_edge(S, T)
            #else:
            #    print((s,t))

    def Rule2(self):
        #Removing self edge from non opaque cells
        
        for I,J in self.digraph.edges(): # removing self edges from topcell that RookField is not trivial
              if I==J:
                i = self.dc.dual(I)
                if i in self.cc.topstar(i):
                    if not set(self.RookField(i))=={0}:
                        self.digraph.remove_edge(I,I)
        
        
        for edge in self.oc(1):
            (s,t) = self.oc.simplex(edge)
            S = self.dc.dual(s)
            T = self.dc.dual(t)
            
            if not self.blowupfringe(S):
                if not self.isOpaque(s): 
                    self.digraph.remove_edge(S,S)
                   # print(s)
                    
            if not self.blowupfringe(T):
                if not self.isOpaque(t): 
                    self.digraph.remove_edge(T,T)
                   # print(t)                   


          
                
    def Rule3(self):
        """
        Apply Rule 3 edge removals
        """
        # print("** Rule 3 ********************************************")
        for s in self.EquilibriumCells():
            #print("=====================")
            #print("Checking equilibrium cell s = " + str(s))
            # print("  topstar(s) = " + str(self.cc.topstar(s)))
            # print("  star(s) = " + str(self.cc.star({s})))
            # print("  fringe in star: " + str([(k, str(self.cc.coordinates(k)), self.cc.rightfringe(k)) for k in self.cc.star({s})]))
            # print("  dim s = " + str(self.cc.cell_dim(s)))
            # print("  Coordinates = " + str(self.cc.coordinates(s)))
            # print("  Barycenter = " + str(self.cc.barycenter(s)))
            # print("  NormalVariables = " + str(self.NormalVariables(s)))
            # print("  TangentVariables = " + str(self.TangentVariables(s)))
            # print("  RegulationMap = " + str(self.RegulationMap(s)))
            # print("  OpacityCycles = " + str(self.OpacityCycles(s)))
            cycles = self.OpacityCycles(s)
            for t in self.cc.topstar(s):
                # print("  Checking top cell t = " + str(t))
                q = self.RelativePositionVector(s,t)
                # print("    RelativePositionVector" + str((s,t)) + " = " + str(q))
                unstable = [ (self.CrossingNumber(q,cycle) <= 
                    (len(cycle)+self.Feedback(cycle))/2 + 1) for cycle in cycles]
                for cycle in cycles:
                    edges = list(zip(cycle[:-1],cycle[1:]))+[(cycle[-1],cycle[0])]
                    # print("    Cycle = " + str(cycle) )
                    # print("      len(cycle) = " + str(len(cycle)))
                    # print("      edges = " + str(edges))
                    # print("      feedback type = " + str(self.Feedback(cycle)))
                    # print("      crossing number = " + str(self.CrossingNumber(q,cycle)))
                    # print("      calc = " + str([ (1 if ((1 if self.network.interaction(i,j) else -1) != q[i]*q[j]) else 0,"delta[i,j]=" + str((1 if self.network.interaction(i,j) else -1)),"q[i]=" + str(q[i]),"q[j] = " + str(q[j]),"i = "+str(i),"j = " + str(j)) for (i,j) in edges]))
                # print("    Unstable = " + str(unstable))
                S = self.dc.dual(s)
                T = self.dc.dual(t)
                if all(unstable):
                    if T in self.digraph.adjacencies(S):
                        self.digraph.remove_edge(T,S)
                        #print("    \033[0;35;48m Removing edge from \033[0;30;48m " + str(t) + " to " + str(s))
                        # print("      barycenter(" + str(t) + ") = " + str(self.cc.barycenter(t)))
                        # print("      barycenter(" + str(s) + ") = " + str(self.cc.barycenter(s)))
                        # print("      CUT(" + str(self.cc.barycenter(t)) + ", " + str(self.cc.barycenter(s)) + ")")
          
                if not any(unstable):
                    if S in self.digraph.adjacencies(T):
                        self.digraph.remove_edge(S,T)
                        #print("    \033[0;35;48m Removing edge from \033[0;30;48m" + str(s) + " to " + str(t))
                        # print("      barycenter(" + str(s) + ") = " + str(self.cc.barycenter(s)))
                        # print("      barycenter(" + str(t) + ") = " + str(self.cc.barycenter(t)))
                        # print("      CUT(" + str(self.cc.barycenter(s)) + ", " + str(self.cc.barycenter(t)) + ")")





    def __init__(self, p, level = 0):
        # initialize complex, graph, supporting data structures
        self.p = p
        self.network = self.p.network()
        self.D = self.network.size()
        self.cc = CubicalComplex([ x + 1 for x in self.network.domains()])
        self.digraph = DirectedAcyclicGraph()
        self.labelling = p.labelling()
        self.limits = self.network.domains()
        self.pv = [1]
        for i in self.limits:
            self.pv.append(self.pv[-1]*i)
        self.oc = OrderComplex(self.cc)
        self.dc = DualComplex(self.oc)
        for cell in self.dc(self.D):
            self.digraph.add_vertex(cell)
            
        # Apply rules
        self.Rule0()
        self.Rule1()
        
        
        if level >= 2:
            self.Rule2()

In [2]:
def dsgrn(pg, pi, method = "blowup"):
    p = pg.parameter(pi)
    if method == "original":
        dg = DSGRN.DomainGraph(p)
        md = DSGRN.MorseDecomposition(dg.digraph())
        mg = DSGRN.MorseGraph(dg, md)
        return DSGRN.DrawGraph(mg)
    elif method == "blowup":
        std = BlowupGraph2(p)
    (dag, fibration)= FlowGradedComplex(std.complex(), std.diagram())
    connection_matrix = ConnectionMatrix(fibration)
    conleyindices = connection_matrix.count()
    fringenode = fibration.value(std.complex().size()-1)
    del conleyindices[fringenode]
    CMG = InducedPoset(dag, lambda v : v in conleyindices)
    return DrawGradedComplex(connection_matrix, CMG)

In [3]:
def ComputeDatabase(netspec, params = None):
    network = DSGRN.Network(netspec)
    pg = DSGRN.ParameterGraph(network)
    if not params:
        params = range(0,pg.size())
    return DSGRN.Table( ['ParameterIndex','Original', 'Blowup'],
                   [[pi,
                     dsgrn(pg, pi, "original"),
                     dsgrn(pg, pi, "blowup")] for pi in params])